In [0]:
import pandas as pd
df = spark.read.csv("/FileStore/tables/Stroke_2010_2019.csv",inferSchema=True, header=True)

In [0]:
pandas_df = df.toPandas()
pandas_df

,Year,LocationAbbr,LocationDesc,GeographicLevel,DataSource,Class,Topic,Data_Value,Data_Value_Unit,Data_Value_Type,Data_Value_Footnote_Symbol,Data_Value_Footnote,Confidence_limit_Low,Confidence_limit_High,StratificationCategory1,Stratification1,StratificationCategory2,Stratification2,StratificationCategory3,Stratification3,LocationID
0,2010,HI,Honolulu,County,NVSS,Cardiovascular Diseases,All stroke,0.1,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",NA,NA,0.1,0.7,Age group,Ages 35-64 years,Race,American Indian/Alaska Native,Sex,Overall,15003
1,2014,HI,Honolulu,County,NVSS,Cardiovascular Diseases,All stroke,0.8,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",NA,NA,0.4,3.3,Age group,Ages 35-64 years,Race,Black (Non-Hispanic),Sex,Overall,15003
2,2014,HI,Hawaii,County,NVSS,Cardiovascular Diseases,All stroke,0.8,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",NA,NA,0.4,6.8,Age group,Ages 35-64 years,Race,Black (Non-Hispanic),Sex,Overall,15001
3,2010,AK,Juneau-Haines-Hoonah-Angoon-Skagway,County,NVSS,Cardiovascular Diseases,All stroke,0.9,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",NA,NA,0.6,1.7,Age group,Ages 35-64 years,Race,Hispanic,Sex,Overall,2902
4,2010,AK,Fairbanks North Star,County,NVSS,Cardiovascular Diseases,All stroke,0.9,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",NA,NA,0.4,2,Age group,Ages 35-64 years,Race,Hispanic,Sex,Overall,2090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501755,2019,NC,Camden,County,NVSS,Cardiovascular Diseases,All stroke,NA,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",~,Value suppressed,NA,NA,Age group,Ages 65 years and older,Race,Black (Non-Hispanic),Sex,Overall,37029
501756,2019,VA,King George,County,NVSS,Cardiovascular Diseases,All stroke,NA,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",~,Value suppressed,NA,NA,Age group,Ages 65 years and older,Race,Asian/Pacific Islander,Sex,Overall,51099
501757,2019,KS,Franklin,County,NVSS,Cardiovascular Diseases,All stroke,NA,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",~,Value suppressed,NA,NA,Age group,Ages 65 years and older,Race,Black (Non-Hispanic),Sex,Overall,20059
501758,2019,PA,Washington,County,NVSS,Cardiovascular Diseases,All stroke,NA,"per 100,000","Age-Standardized, Spatiotemporally Smoothed Rate",~,Value suppressed,NA,NA,Age group,Ages 65 years and older,Race,American Indian/Alaska Native,Sex,Overall,42125


In [0]:
for column in pandas_df.columns:
    value_counts = pandas_df[column].value_counts()
    print(f"Value Counts for {column}:\n{value_counts}\n")

Value Counts for Year:
2010    50176
2014    50176
2011    50176
2012    50176
2013    50176
2015    50176
2016    50176
2017    50176
2018    50176
2019    50176
Name: Year, dtype: int64

Value Counts for LocationAbbr:
TX    40640
GA    25440
VA    21280
KY    19200
MO    18400
KS    16800
IL    16320
NC    16000
IA    15840
TN    15200
NE    14880
IN    14720
OH    14080
MN    13920
MI    13280
MS    13120
OK    12320
AR    12000
WI    11520
FL    10720
AL    10720
PA    10720
SD    10560
LA    10240
CO    10240
NY     9920
CA     9280
MT     8960
WV     8800
ND     8480
SC     7360
ID     7040
WA     6240
OR     5760
NM     5280
UT     4640
MD     3840
AK     3680
WY     3680
NJ     3360
NV     2720
ME     2560
AZ     2400
MA     2240
VT     2240
NH     1600
CT     1280
RI      800
HI      800
DE      480
DC      160
Name: LocationAbbr, dtype: int64

Value Counts for LocationDesc:
Washington     4960
Jefferson      4160
Franklin       4160
Lincoln        3840
Jackson        3840
   

In [0]:
pandas_df["Data_Value"] = pd.to_numeric(pandas_df["Data_Value"], errors='coerce')
pandas_df = pandas_df[pandas_df['Data_Value'].notna()]

These variables have only 1 value, remove them: 
GeographicLevel, DataSource, 

In [0]:
pandas_df = pandas_df[['Year','LocationAbbr','Data_Value', 'Stratification1', 'Stratification2', 'Stratification3']]

In [0]:
pandas_df = pandas_df.rename(columns={'Stratification1': 'Age', 'Stratification2': 'Race', 'Stratification3': 'Gender', 'LocationAbbr': 'State','Data_Value': 'Mortality_Rate'})
pandas_df

,Year,State,Mortality_Rate,Age,Race,Gender
0,2010,HI,0.1,Ages 35-64 years,American Indian/Alaska Native,Overall
1,2014,HI,0.8,Ages 35-64 years,Black (Non-Hispanic),Overall
2,2014,HI,0.8,Ages 35-64 years,Black (Non-Hispanic),Overall
3,2010,AK,0.9,Ages 35-64 years,Hispanic,Overall
4,2010,AK,0.9,Ages 35-64 years,Hispanic,Overall
...,...,...,...,...,...,...
296095,2019,MT,9.9,Ages 35-64 years,White,Overall
296096,2019,FL,9.9,Ages 35-64 years,Hispanic,Overall
296097,2019,WY,9.9,Ages 35-64 years,Overall,Women
296098,2019,PA,9.9,Ages 35-64 years,Hispanic,Overall


In [0]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
 
from gensim.models.fasttext import FastText
 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
 
# Lemmatization
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()

In [0]:
articles = spark.read.csv("/FileStore/tables/voat_stroke_articles3.csv",inferSchema=True, header=True)
text = articles.toPandas()

In [0]:
text['combined_text'] = text['lead_paragraph'] + ' ' + text['abstract']
text

,_id,abstract,snippet,lead_paragraph,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,pub_date,section_name,word_count,Year,State,Hits,combined_text
0,https://www.nytimes.com/interactive/2019/06/19...,Air pollution has improved dramatically over t...,Air pollution has improved dramatically over t...,Air pollution has improved dramatically over t...,"America’s Skies Have Gotten Clearer, but Milli...",None,None,None,None,None,None,2019-06-19 14:40:54,Climate,0,2019,Wyoming,3,Air pollution has improved dramatically over t...
1,https://www.nytimes.com/2019/06/16/fashion/wed...,All of the weddings right here on one handy pa...,All of the weddings right here on one handy pa...,Julius Alan Schorzman and Louis Edward Morton ...,This Week’s Wedding Announcements,None,None,None,None,None,None,2019-06-16 04:30:04,Fashion & Style,6636,2019,Wyoming,3,Julius Alan Schorzman and Louis Edward Morton ...
2,https://www.nytimes.com/2019/06/20/sports/nba-...,Zion Williamson was taken No. 1 overall as exp...,Zion Williamson was taken No. 1 overall as exp...,The first three picks went exactly as everyone...,2019 N.B.A. Draft Results: Analysis of Every P...,None,None,Pelicans Get Williamson; Grizzlies Take Morant,None,None,None,2019-06-20 22:46:43,Sports,7484,2019,Wyoming,3,The first three picks went exactly as everyone...
3,https://www.nytimes.com/2019/07/08/well/live/s...,A young woman’s struggle to help a husband who...,A young woman’s struggle to help a husband who...,Kelly Baxter was 36 years old and had just mov...,The Challenge of Caring for a Stroke Patient,Personal health,None,The Challenge of Caring for a Stroke Patient,None,None,None,2019-07-08 09:00:03,Well,1140,2019,Wisconsin,15,Kelly Baxter was 36 years old and had just mov...
4,https://www.nytimes.com/interactive/2019/books...,Five new books take us to far-flung places inc...,Five new books take us to far-flung places inc...,Five new books take us to far-flung places inc...,Crossing the Globe to Understand Other People ...,None,None,None,None,None,None,2019-11-26 20:42:50,Books,0,2019,Wisconsin,15,Five new books take us to far-flung places inc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5118,https://www.nytimes.com/2010/06/29/health/29ge...,Seventy million Americans will have turned 65 ...,Seventy million Americans will have turned 65 ...,With a nudge from the new health care law and ...,Preparing More Care of Elderly,The New Landscape,None,Preparing More Care Of Elderly,None,None,None,2010-06-28 21:01:50,Health,1737,2010,Alabama,7,With a nudge from the new health care law and ...
5119,https://www.nytimes.com/2010/10/29/health/poli...,Federal regulators seem to have heightened the...,Federal regulators seem to have heightened the...,The Food and Drug Administration rejected ano...,"F.D.A. Rejects Qnexa, a Third Weight-Loss Drug",None,None,"With Safety Concerns Growing, F.D.A. Rejects A...",None,None,None,2010-10-29 00:58:20,Health,1383,2010,Alabama,7,The Food and Drug Administration rejected ano...
5120,https://www.nytimes.com/2010/02/28/magazine/28...,James Fields is an African-American Democratic...,James Fields is an African-American Democratic...,"SEVERAL DAYS A WEEK, a tall, broad-shouldered...",Race in the South in the Age of Obama,None,None,The Visible Man,None,None,None,2010-02-25 15:41:57,Magazine,8159,2010,Alabama,7,"SEVERAL DAYS A WEEK, a tall, broad-shouldered..."
5121,https://prescriptions.blogs.nytimes.com/2010/0...,The House convenes for a historic vote on heal...,The House convenes for a historic vote on heal...,12:31 a.m. |A Long Day,Live Blogging the House Vote,Prescriptions,None,None,None,None,None,2010-03-21 17:04:23,Health,10942,2010,Alabama,7,12:31 a.m. |A Long Day The House convenes for ...


In [0]:
# State abbreviation mapping for all 50 states and Washington D.C.
state_abbreviations = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
    "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA",
    "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA",
    "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO",
    "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ",
    "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH",
    "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT",
    "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY",
    "District of Columbia": "DC"
}

# Create a new column 'StateAbbreviation' based on the 'State' column
text['StateAbbr'] = text['State'].map(state_abbreviations)
text

,_id,abstract,snippet,lead_paragraph,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,pub_date,section_name,word_count,Year,State,Hits,combined_text,StateAbbr
0,https://www.nytimes.com/interactive/2019/06/19...,Air pollution has improved dramatically over t...,Air pollution has improved dramatically over t...,Air pollution has improved dramatically over t...,"America’s Skies Have Gotten Clearer, but Milli...",None,None,None,None,None,None,2019-06-19 14:40:54,Climate,0,2019,Wyoming,3,Air pollution has improved dramatically over t...,WY
1,https://www.nytimes.com/2019/06/16/fashion/wed...,All of the weddings right here on one handy pa...,All of the weddings right here on one handy pa...,Julius Alan Schorzman and Louis Edward Morton ...,This Week’s Wedding Announcements,None,None,None,None,None,None,2019-06-16 04:30:04,Fashion & Style,6636,2019,Wyoming,3,Julius Alan Schorzman and Louis Edward Morton ...,WY
2,https://www.nytimes.com/2019/06/20/sports/nba-...,Zion Williamson was taken No. 1 overall as exp...,Zion Williamson was taken No. 1 overall as exp...,The first three picks went exactly as everyone...,2019 N.B.A. Draft Results: Analysis of Every P...,None,None,Pelicans Get Williamson; Grizzlies Take Morant,None,None,None,2019-06-20 22:46:43,Sports,7484,2019,Wyoming,3,The first three picks went exactly as everyone...,WY
3,https://www.nytimes.com/2019/07/08/well/live/s...,A young woman’s struggle to help a husband who...,A young woman’s struggle to help a husband who...,Kelly Baxter was 36 years old and had just mov...,The Challenge of Caring for a Stroke Patient,Personal health,None,The Challenge of Caring for a Stroke Patient,None,None,None,2019-07-08 09:00:03,Well,1140,2019,Wisconsin,15,Kelly Baxter was 36 years old and had just mov...,WI
4,https://www.nytimes.com/interactive/2019/books...,Five new books take us to far-flung places inc...,Five new books take us to far-flung places inc...,Five new books take us to far-flung places inc...,Crossing the Globe to Understand Other People ...,None,None,None,None,None,None,2019-11-26 20:42:50,Books,0,2019,Wisconsin,15,Five new books take us to far-flung places inc...,WI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5118,https://www.nytimes.com/2010/06/29/health/29ge...,Seventy million Americans will have turned 65 ...,Seventy million Americans will have turned 65 ...,With a nudge from the new health care law and ...,Preparing More Care of Elderly,The New Landscape,None,Preparing More Care Of Elderly,None,None,None,2010-06-28 21:01:50,Health,1737,2010,Alabama,7,With a nudge from the new health care law and ...,AL
5119,https://www.nytimes.com/2010/10/29/health/poli...,Federal regulators seem to have heightened the...,Federal regulators seem to have heightened the...,The Food and Drug Administration rejected ano...,"F.D.A. Rejects Qnexa, a Third Weight-Loss Drug",None,None,"With Safety Concerns Growing, F.D.A. Rejects A...",None,None,None,2010-10-29 00:58:20,Health,1383,2010,Alabama,7,The Food and Drug Administration rejected ano...,AL
5120,https://www.nytimes.com/2010/02/28/magazine/28...,James Fields is an African-American Democratic...,James Fields is an African-American Democratic...,"SEVERAL DAYS A WEEK, a tall, broad-shouldered...",Race in the South in the Age of Obama,None,None,The Visible Man,None,None,None,2010-02-25 15:41:57,Magazine,8159,2010,Alabama,7,"SEVERAL DAYS A WEEK, a tall, broad-shouldered...",AL
5121,https://prescriptions.blogs.nytimes.com/2010/0...,The House convenes for a historic vote on heal...,The House convenes for a historic vote on heal...,12:31 a.m. |A Long Day,Live Blogging the House Vote,Prescriptions,None,None,None,None,None,2010-03-21 17:04:23,Health,10942,2010,Alabama,7,12:31 a.m. |A Long Day The House convenes for ...,AL


In [0]:
value_counts = text['section_name'].value_counts()
print(f"Value Counts for {'section_name'}:\n{value_counts}\n")

Value Counts for section_name:
Health                  963
U.S.                    795
Magazine                464
Opinion                 435
New York                265
Business Day            250
Books                   225
Sports                  206
Arts                    196
Well                    173
Fashion & Style         139
World                   117
The Upshot               90
Briefing                 89
The Learning Network     87
T Brand                  66
Obituaries               65
Science                  60
Theater                  49
Your Money               41
Climate                  39
Booming                  34
NYT Now                  29
Food                     26
Education                25
Multimedia/Photos        24
Movies                   22
Style                    21
Technology               19
Travel                   17
Times Insider            14
Blogs                    13
T Magazine               12
Reader Center             9
Sunday Review    

In [0]:
text['Section'] = text['section_name'].replace({
    'Magazine': 'Magazine + Opinion',
    'Opinion': 'Magazine + Opinion',
    'U.S.': 'U.S.',
    'Health': 'Health',
})

In [0]:
text['Section'].where(text['Section'].isin(['Health', 'U.S.', 'Magazine + Opinion']), 'Others', inplace=True)
text

,_id,abstract,snippet,lead_paragraph,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,pub_date,section_name,word_count,Year,State,Hits,combined_text,StateAbbr,Section
0,https://www.nytimes.com/interactive/2019/06/19...,Air pollution has improved dramatically over t...,Air pollution has improved dramatically over t...,Air pollution has improved dramatically over t...,"America’s Skies Have Gotten Clearer, but Milli...",None,None,None,None,None,None,2019-06-19 14:40:54,Climate,0,2019,Wyoming,3,Air pollution has improved dramatically over t...,WY,Others
1,https://www.nytimes.com/2019/06/16/fashion/wed...,All of the weddings right here on one handy pa...,All of the weddings right here on one handy pa...,Julius Alan Schorzman and Louis Edward Morton ...,This Week’s Wedding Announcements,None,None,None,None,None,None,2019-06-16 04:30:04,Fashion & Style,6636,2019,Wyoming,3,Julius Alan Schorzman and Louis Edward Morton ...,WY,Others
2,https://www.nytimes.com/2019/06/20/sports/nba-...,Zion Williamson was taken No. 1 overall as exp...,Zion Williamson was taken No. 1 overall as exp...,The first three picks went exactly as everyone...,2019 N.B.A. Draft Results: Analysis of Every P...,None,None,Pelicans Get Williamson; Grizzlies Take Morant,None,None,None,2019-06-20 22:46:43,Sports,7484,2019,Wyoming,3,The first three picks went exactly as everyone...,WY,Others
3,https://www.nytimes.com/2019/07/08/well/live/s...,A young woman’s struggle to help a husband who...,A young woman’s struggle to help a husband who...,Kelly Baxter was 36 years old and had just mov...,The Challenge of Caring for a Stroke Patient,Personal health,None,The Challenge of Caring for a Stroke Patient,None,None,None,2019-07-08 09:00:03,Well,1140,2019,Wisconsin,15,Kelly Baxter was 36 years old and had just mov...,WI,Others
4,https://www.nytimes.com/interactive/2019/books...,Five new books take us to far-flung places inc...,Five new books take us to far-flung places inc...,Five new books take us to far-flung places inc...,Crossing the Globe to Understand Other People ...,None,None,None,None,None,None,2019-11-26 20:42:50,Books,0,2019,Wisconsin,15,Five new books take us to far-flung places inc...,WI,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5118,https://www.nytimes.com/2010/06/29/health/29ge...,Seventy million Americans will have turned 65 ...,Seventy million Americans will have turned 65 ...,With a nudge from the new health care law and ...,Preparing More Care of Elderly,The New Landscape,None,Preparing More Care Of Elderly,None,None,None,2010-06-28 21:01:50,Health,1737,2010,Alabama,7,With a nudge from the new health care law and ...,AL,Health
5119,https://www.nytimes.com/2010/10/29/health/poli...,Federal regulators seem to have heightened the...,Federal regulators seem to have heightened the...,The Food and Drug Administration rejected ano...,"F.D.A. Rejects Qnexa, a Third Weight-Loss Drug",None,None,"With Safety Concerns Growing, F.D.A. Rejects A...",None,None,None,2010-10-29 00:58:20,Health,1383,2010,Alabama,7,The Food and Drug Administration rejected ano...,AL,Health
5120,https://www.nytimes.com/2010/02/28/magazine/28...,James Fields is an African-American Democratic...,James Fields is an African-American Democratic...,"SEVERAL DAYS A WEEK, a tall, broad-shouldered...",Race in the South in the Age of Obama,None,None,The Visible Man,None,None,None,2010-02-25 15:41:57,Magazine,8159,2010,Alabama,7,"SEVERAL DAYS A WEEK, a tall, broad-shouldered...",AL,Magazine + Opinion
5121,https://prescriptions.blogs.nytimes.com/2010/0...,The House convenes for a historic vote on heal...,The House convenes for a historic vote on heal...,12:31 a.m. |A Long Day,Live Blogging the House Vote,Prescriptions,None,None,None,None,None,2010-03-21 17:04:23,Health,10942,2010,Alabama,7,12:31 a.m. |A Long Day The House convenes for ...,AL,Health


In [0]:
text = text[['abstract', 'lead_paragraph', 'Section',	'word_count', 'Year', 'StateAbbr',	'Hits', 'combined_text']]
text

,abstract,lead_paragraph,Section,word_count,Year,StateAbbr,Hits,combined_text
0,Air pollution has improved dramatically over t...,Air pollution has improved dramatically over t...,Others,0,2019,WY,3,Air pollution has improved dramatically over t...
1,All of the weddings right here on one handy pa...,Julius Alan Schorzman and Louis Edward Morton ...,Others,6636,2019,WY,3,Julius Alan Schorzman and Louis Edward Morton ...
2,Zion Williamson was taken No. 1 overall as exp...,The first three picks went exactly as everyone...,Others,7484,2019,WY,3,The first three picks went exactly as everyone...
3,A young woman’s struggle to help a husband who...,Kelly Baxter was 36 years old and had just mov...,Others,1140,2019,WI,15,Kelly Baxter was 36 years old and had just mov...
4,Five new books take us to far-flung places inc...,Five new books take us to far-flung places inc...,Others,0,2019,WI,15,Five new books take us to far-flung places inc...
...,...,...,...,...,...,...,...,...
5118,Seventy million Americans will have turned 65 ...,With a nudge from the new health care law and ...,Health,1737,2010,AL,7,With a nudge from the new health care law and ...
5119,Federal regulators seem to have heightened the...,The Food and Drug Administration rejected ano...,Health,1383,2010,AL,7,The Food and Drug Administration rejected ano...
5120,James Fields is an African-American Democratic...,"SEVERAL DAYS A WEEK, a tall, broad-shouldered...",Magazine + Opinion,8159,2010,AL,7,"SEVERAL DAYS A WEEK, a tall, broad-shouldered..."
5121,The House convenes for a historic vote on heal...,12:31 a.m. |A Long Day,Health,10942,2010,AL,7,12:31 a.m. |A Long Day The House convenes for ...


In [0]:
# import NLTK, and download vader_lexicon
import nltk
nltk.download('vader_lexicon')
# import nltk vader library
from nltk.sentiment.vader import SentimentIntensityAnalyzer
    
# initiate an analyzer
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
corpus = text['lead_paragraph']
corpus2 = text['abstract']

corpus = corpus.astype(str)
corpus2 = corpus2.astype(str)

lead_neg = []
lead_comp = []
abs_neg = []
abs_comp = []

# iterate through each sentence in corpus

for sentence in corpus:
    ss = sia.polarity_scores(sentence)  
    lead_neg.append(ss['neg'])
    lead_comp.append(ss['compound'])

for sentence in corpus2:
    ss = sia.polarity_scores(sentence)  
    abs_neg.append(ss['neg'])
    abs_comp.append(ss['compound'])


In [0]:
text = text.assign(lead_neg = lead_neg, lead_compound = lead_comp, abs_neg = abs_neg, abs_compound = abs_comp)

In [0]:
text

,abstract,lead_paragraph,Section,word_count,Year,StateAbbr,Hits,combined_text,lead_neg,lead_compound,abs_neg,abs_compound
0,Air pollution has improved dramatically over t...,Air pollution has improved dramatically over t...,Others,0,2019,WY,3,Air pollution has improved dramatically over t...,0.103,-0.4767,0.103,-0.4767
1,All of the weddings right here on one handy pa...,Julius Alan Schorzman and Louis Edward Morton ...,Others,6636,2019,WY,3,Julius Alan Schorzman and Louis Edward Morton ...,0.000,0.2023,0.000,0.0000
2,Zion Williamson was taken No. 1 overall as exp...,The first three picks went exactly as everyone...,Others,7484,2019,WY,3,The first three picks went exactly as everyone...,0.000,0.0000,0.104,-0.0772
3,A young woman’s struggle to help a husband who...,Kelly Baxter was 36 years old and had just mov...,Others,1140,2019,WI,15,Kelly Baxter was 36 years old and had just mov...,0.251,-0.8402,0.164,0.1027
4,Five new books take us to far-flung places inc...,Five new books take us to far-flung places inc...,Others,0,2019,WI,15,Five new books take us to far-flung places inc...,0.000,0.0000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
5118,Seventy million Americans will have turned 65 ...,With a nudge from the new health care law and ...,Health,1737,2010,AL,7,With a nudge from the new health care law and ...,0.136,0.2023,0.099,-0.0516
5119,Federal regulators seem to have heightened the...,The Food and Drug Administration rejected ano...,Health,1383,2010,AL,7,The Food and Drug Administration rejected ano...,0.177,-0.7003,0.130,-0.2732
5120,James Fields is an African-American Democratic...,"SEVERAL DAYS A WEEK, a tall, broad-shouldered...",Magazine + Opinion,8159,2010,AL,7,"SEVERAL DAYS A WEEK, a tall, broad-shouldered...",0.000,0.7696,0.057,-0.1280
5121,The House convenes for a historic vote on heal...,12:31 a.m. |A Long Day,Health,10942,2010,AL,7,12:31 a.m. |A Long Day The House convenes for ...,0.000,0.0000,0.000,0.4939


In [0]:
# Load the regular expression library
import re

# Remove punctuation and convert to lowercase
text['paper_text_processed'] = text['combined_text'].apply(lambda x: re.sub('[,\.!?]', '', str(x)))
text['paper_text_processed'] = text['paper_text_processed'].apply(lambda x: x.lower())

# Print out the first rows of papers
text.head(5)


,abstract,lead_paragraph,Section,word_count,Year,StateAbbr,Hits,combined_text,lead_neg,lead_compound,abs_neg,abs_compound,paper_text_processed
0,Air pollution has improved dramatically over t...,Air pollution has improved dramatically over t...,Others,0,2019,WY,3,Air pollution has improved dramatically over t...,0.103,-0.4767,0.103,-0.4767,air pollution has improved dramatically over t...
1,All of the weddings right here on one handy pa...,Julius Alan Schorzman and Louis Edward Morton ...,Others,6636,2019,WY,3,Julius Alan Schorzman and Louis Edward Morton ...,0.000,0.2023,0.000,0.0000,julius alan schorzman and louis edward morton ...
2,Zion Williamson was taken No. 1 overall as exp...,The first three picks went exactly as everyone...,Others,7484,2019,WY,3,The first three picks went exactly as everyone...,0.000,0.0000,0.104,-0.0772,the first three picks went exactly as everyone...
3,A young woman’s struggle to help a husband who...,Kelly Baxter was 36 years old and had just mov...,Others,1140,2019,WI,15,Kelly Baxter was 36 years old and had just mov...,0.251,-0.8402,0.164,0.1027,kelly baxter was 36 years old and had just mov...
4,Five new books take us to far-flung places inc...,Five new books take us to far-flung places inc...,Others,0,2019,WI,15,Five new books take us to far-flung places inc...,0.000,0.0000,0.000,0.0000,five new books take us to far-flung places inc...


In [0]:
word_tokenizer = nltk.WordPunctTokenizer()
word_tokens = [word_tokenizer.tokenize(sent) for sent in tqdm(text['paper_text_processed'])]

# Tokenize the processed text
text['word_tokens'] = text['paper_text_processed'].apply(lambda x: word_tokenizer.tokenize(x))

100%|██████████| 5123/5123 [00:00<00:00, 31633.76it/s]


In [0]:
import time
from gensim.models import FastText

# Defining values for parameters
embedding_size = 300
window_size = 5
min_word = 5
down_sampling = 1e-2

# Your FastText model creation code
fast_Text_model = FastText(word_tokens,
                           window=window_size,
                           min_count=min_word,
                           sample=down_sampling,
                           workers=4,
                           sg=1,
                           epochs=100)  # Use epochs instead of iter

text['embedding'] = text['word_tokens'].apply(lambda tokens: np.mean([fast_Text_model.wv[token] for token in tokens if token in fast_Text_model.wv] or [np.zeros(embedding_size)], axis=0))

In [0]:
# Calculate the average for each list in the 'embedding' column
text['average_embedding'] = text['embedding'].apply(lambda emb_list: np.mean(emb_list, axis=0))

In [0]:
text = text[['Year', 'StateAbbr', 'Section', 'word_count', 'Hits', 'lead_neg',	'lead_compound', 'abs_neg',	'abs_compound', 'average_embedding']]
text

,Year,StateAbbr,Section,word_count,Hits,lead_neg,lead_compound,abs_neg,abs_compound,average_embedding
0,2019,WY,Others,0,3,0.103,-0.4767,0.103,-0.4767,-0.016566
1,2019,WY,Others,6636,3,0.000,0.2023,0.000,0.0000,-0.005678
2,2019,WY,Others,7484,3,0.000,0.0000,0.104,-0.0772,-0.008237
3,2019,WI,Others,1140,15,0.251,-0.8402,0.164,0.1027,-0.008439
4,2019,WI,Others,0,15,0.000,0.0000,0.000,0.0000,-0.008383
...,...,...,...,...,...,...,...,...,...,...
5118,2010,AL,Health,1737,7,0.136,0.2023,0.099,-0.0516,-0.017206
5119,2010,AL,Health,1383,7,0.177,-0.7003,0.130,-0.2732,-0.004767
5120,2010,AL,Magazine + Opinion,8159,7,0.000,0.7696,0.057,-0.1280,-0.011460
5121,2010,AL,Health,10942,7,0.000,0.0000,0.000,0.4939,-0.037272


In [0]:
summary_table = text.groupby(['StateAbbr', 'Year']).mean().reset_index()
summary_table

,StateAbbr,Year,word_count,Hits,lead_neg,lead_compound,abs_neg,abs_compound,average_embedding
0,AK,2010,1740.250000,4.0,0.097250,-0.289025,0.052750,0.207200,-0.015775
1,AK,2011,1048.000000,2.0,0.083500,-0.359200,0.082500,-0.218200,-0.004370
2,AK,2012,14821.250000,4.0,0.016750,0.143100,0.057750,-0.037400,-0.009749
3,AK,2013,1281.500000,2.0,0.159500,-0.489850,0.174500,-0.354800,-0.017311
4,AK,2014,583.333333,3.0,0.060000,0.275333,0.039333,0.322300,0.005115
...,...,...,...,...,...,...,...,...,...
482,WY,2013,1077.500000,2.0,0.159500,-0.489850,0.108000,-0.148000,-0.007204
483,WY,2014,362.000000,2.0,0.090000,0.058200,0.028500,0.346850,0.008543
484,WY,2015,1649.666667,3.0,0.103333,-0.055033,0.084000,-0.092967,-0.030396
485,WY,2016,2787.500000,2.0,0.041500,-0.105350,0.145000,-0.181850,0.003743


In [0]:
merged_table = pd.merge(pandas_df, summary_table, left_on=['State', 'Year'], right_on=['StateAbbr', 'Year'], how='left')
merged_table

,Year,State,Mortality_Rate,Age,Race,Gender,StateAbbr,word_count,Hits,lead_neg,lead_compound,abs_neg,abs_compound,average_embedding
0,2010,HI,0.1,Ages 35-64 years,American Indian/Alaska Native,Overall,HI,2453.000000,1.0,0.191000,-0.931900,0.117000,0.318200,-0.005470
1,2014,HI,0.8,Ages 35-64 years,Black (Non-Hispanic),Overall,HI,8498.000000,4.0,0.078500,-0.161650,0.018000,0.212975,-0.019061
2,2014,HI,0.8,Ages 35-64 years,Black (Non-Hispanic),Overall,HI,8498.000000,4.0,0.078500,-0.161650,0.018000,0.212975,-0.019061
3,2010,AK,0.9,Ages 35-64 years,Hispanic,Overall,AK,1740.250000,4.0,0.097250,-0.289025,0.052750,0.207200,-0.015775
4,2010,AK,0.9,Ages 35-64 years,Hispanic,Overall,AK,1740.250000,4.0,0.097250,-0.289025,0.052750,0.207200,-0.015775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296095,2019,MT,9.9,Ages 35-64 years,White,Overall,MT,744.000000,3.0,0.054000,-0.022133,0.076667,-0.305700,-0.010886
296096,2019,FL,9.9,Ages 35-64 years,Hispanic,Overall,FL,2077.913043,23.0,0.057957,-0.059883,0.072348,-0.048043,-0.010302
296097,2019,WY,9.9,Ages 35-64 years,Overall,Women,WY,4706.666667,3.0,0.034333,-0.091467,0.069000,-0.184633,-0.010160
296098,2019,PA,9.9,Ages 35-64 years,Hispanic,Overall,PA,2112.736842,19.0,0.082000,-0.177479,0.070000,-0.048911,-0.012801


In [0]:
merged_table[merged_table['State'] == 'DC']

,Year,State,Mortality_Rate,Age,Race,Gender,StateAbbr,word_count,Hits,lead_neg,lead_compound,abs_neg,abs_compound,average_embedding
697,2016,DC,10.0,Ages 35-64 years,Hispanic,Overall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1417,2013,DC,10.1,Ages 35-64 years,Hispanic,Overall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1618,2015,DC,10.1,Ages 35-64 years,Hispanic,Overall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2160,2010,DC,10.2,Ages 35-64 years,Asian/Pacific Islander,Overall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2558,2013,DC,10.2,Ages 35-64 years,Asian/Pacific Islander,Overall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290552,2012,DC,9.4,Ages 35-64 years,American Indian/Alaska Native,Overall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291408,2011,DC,9.5,Ages 35-64 years,Hispanic,Overall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291641,2014,DC,9.5,Ages 35-64 years,Hispanic,Overall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
291830,2016,DC,9.5,Ages 35-64 years,Asian/Pacific Islander,Overall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
merged_table = merged_table[merged_table['StateAbbr'].notna()]
merged_table.drop(['StateAbbr'], axis=1, inplace=True)

In [0]:
merged_table

,Year,State,Mortality_Rate,Age,Race,Gender,word_count,Hits,lead_neg,lead_compound,abs_neg,abs_compound,average_embedding
0,2010,HI,0.1,Ages 35-64 years,American Indian/Alaska Native,Overall,2453.000000,1.0,0.191000,-0.931900,0.117000,0.318200,-0.005470
1,2014,HI,0.8,Ages 35-64 years,Black (Non-Hispanic),Overall,8498.000000,4.0,0.078500,-0.161650,0.018000,0.212975,-0.019061
2,2014,HI,0.8,Ages 35-64 years,Black (Non-Hispanic),Overall,8498.000000,4.0,0.078500,-0.161650,0.018000,0.212975,-0.019061
3,2010,AK,0.9,Ages 35-64 years,Hispanic,Overall,1740.250000,4.0,0.097250,-0.289025,0.052750,0.207200,-0.015775
4,2010,AK,0.9,Ages 35-64 years,Hispanic,Overall,1740.250000,4.0,0.097250,-0.289025,0.052750,0.207200,-0.015775
...,...,...,...,...,...,...,...,...,...,...,...,...,...
296095,2019,MT,9.9,Ages 35-64 years,White,Overall,744.000000,3.0,0.054000,-0.022133,0.076667,-0.305700,-0.010886
296096,2019,FL,9.9,Ages 35-64 years,Hispanic,Overall,2077.913043,23.0,0.057957,-0.059883,0.072348,-0.048043,-0.010302
296097,2019,WY,9.9,Ages 35-64 years,Overall,Women,4706.666667,3.0,0.034333,-0.091467,0.069000,-0.184633,-0.010160
296098,2019,PA,9.9,Ages 35-64 years,Hispanic,Overall,2112.736842,19.0,0.082000,-0.177479,0.070000,-0.048911,-0.012801


In [0]:
merged_table.isna().sum()

Year                 0
State                0
Mortality_Rate       0
Age                  0
Race                 0
Gender               0
word_count           0
Hits                 0
lead_neg             0
lead_compound        0
abs_neg              0
abs_compound         0
average_embedding    0
dtype: int64

In [0]:
spark_df = spark.createDataFrame(merged_table)

In [0]:
(trainDF, testDF) = spark_df.randomSplit([.8, .2], seed=42)

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

In [0]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

categoricalCols = [field for (field, dataType) in trainDF.dtypes if dataType == "string"]
indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=indexOutputCols, handleInvalid="skip")
oheEncoder = OneHotEncoder(inputCols=indexOutputCols, outputCols=oheOutputCols)

In [0]:
numericCols = [field for (field, dataType) in trainDF.dtypes if (dataType in ["double", "float", "int"]) and (field != "Mortality_Rate")]
assemblerInputs = oheOutputCols + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

In [0]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="Mortality_Rate", featuresCol="features")

In [0]:
from pyspark.ml import Pipeline

stages = [stringIndexer, oheEncoder, vecAssembler, lr]
pipeline = Pipeline(stages=stages)

pipelineModel = pipeline.fit(trainDF)
predDF = pipelineModel.transform(testDF)

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

regressionEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Mortality_Rate", metricName="rmse")

rmse = regressionEvaluator.evaluate(predDF)
r2 = regressionEvaluator.setMetricName("r2").evaluate(predDF)
print(f"RMSE is {rmse}")
print(f"R2 is {r2}")

RMSE is 36.91377823073908
R2 is 0.926611117372046


In [0]:
output_col = vecAssembler.getOutputCol()

# Get the feature names and indices from the DataFrame's schema metadata
metadata = pipelineModel.transform(trainDF).select(output_col).schema[0].metadata["ml_attr"]["attrs"]
numeric_metadata = metadata["numeric"]
binary_metadata = metadata["binary"]

# Create a dictionary to map feature indices to names
index_to_name = {entry["idx"]: entry["name"] for entry in numeric_metadata + binary_metadata}

# Assuming lr is your LinearRegression model
coefficients = pipelineModel.stages[-1].coefficients

# Create a dictionary to map feature names to coefficients
coefficients_dict = {index_to_name[i]: coefficient for i, coefficient in enumerate(coefficients)}

# Print coefficients
for feature, coefficient in coefficients_dict.items():
    print(f"{feature}: {coefficient}")

StateOHE_TX: 13.819480661646152
StateOHE_GA: 18.242452721915456
StateOHE_VA: 11.013089701481837
StateOHE_NC: 14.503672314519926
StateOHE_KY: 21.144292486408116
StateOHE_MO: 11.433200253649456
StateOHE_IL: 4.812395955899906
StateOHE_TN: 24.273110413165274
StateOHE_OH: 10.56118174409027
StateOHE_IN: 14.05963600633638
StateOHE_IA: -11.149146541827811
StateOHE_FL: -8.48744526840803
StateOHE_MI: 4.721337930418687
StateOHE_MS: 29.304645859571675
StateOHE_KS: 2.784848811500785
StateOHE_MN: -10.64208339497113
StateOHE_CA: 1.6550917888242949
StateOHE_AR: 27.848538054491083
StateOHE_NY: -26.916513927573796
StateOHE_AL: 32.92515757711882
StateOHE_LA: 18.27552354397769
StateOHE_PA: -5.876046861379722
StateOHE_OK: 9.772042491174489
StateOHE_WI: -4.590410199487139
StateOHE_CO: -16.236876692973325
StateOHE_SC: 27.157452972664338
StateOHE_NE: -10.368517057013566
StateOHE_WV: 11.514954636125024
StateOHE_WA: -0.3782073777179727
StateOHE_NM: -17.786618906378255
StateOHE_ID: 0.8076978216952065
StateOHE_MT

In [0]:
from pyspark.ml.regression import RandomForestRegressor

# Random Forest
rf = RandomForestRegressor(labelCol="Mortality_Rate", featuresCol="features", numTrees=100, seed=42)

# Update the stages in the pipeline
stages = [stringIndexer, oheEncoder, vecAssembler, rf]
pipeline = Pipeline(stages=stages)

# Fit the pipeline
pipelineModel = pipeline.fit(trainDF)

# Make predictions on the test set
predDF_rf = pipelineModel.transform(testDF)

In [0]:
# Evaluate the model
regressionEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Mortality_Rate", metricName="rmse")

rmse = regressionEvaluator.evaluate(predDF_rf)
r2 = regressionEvaluator.setMetricName("r2").evaluate(predDF_rf)
print(f"RMSE is {rmse}")
print(f"R2 is {r2}")


RMSE is 39.525210862679124
R2 is 0.9158601624847638
Feature Importances (Random Forest):
StateOHE: 0.0017201886073924403
AgeOHE: 0.0015962860117875132
RaceOHE: 0.0
GenderOHE: 0.000328485031964128
Year: 0.004026687846207561
word_count: 0.0
Hits: 0.0
lead_neg: 0.004974871860181619
lead_compound: 0.0
abs_neg: 0.0004090951065779022
abs_compound: 0.004576927350921641
average_embedding: 0.004075981014742925


In [0]:
from pyspark.ml.regression import GBTRegressor

# Update the regressor
gbt = GBTRegressor(labelCol="Mortality_Rate", featuresCol="features")

# Update the stages in the pipeline
stages = [stringIndexer, oheEncoder, vecAssembler, gbt]
pipeline = Pipeline(stages=stages)

pipelineModel = pipeline.fit(trainDF)

predDF_gbt = pipelineModel.transform(testDF)

regressionEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Mortality_Rate", metricName="rmse")

rmse = regressionEvaluator.evaluate(predDF_gbt)
r2 = regressionEvaluator.setMetricName("r2").evaluate(predDF_gbt)
print(f"RMSE is {rmse}")
print(f"R2 is {r2}")


RMSE is 33.581209783210134
R2 is 0.9392640325639816


In [0]:
feature_importances = pipelineModel.stages[-1].featureImportances
feature_importances

SparseVector(65, {0: 0.0017, 1: 0.0016, 3: 0.0003, 4: 0.004, 7: 0.005, 9: 0.0004, 10: 0.0046, 11: 0.0041, 13: 0.0056, 15: 0.0042, 16: 0.0001, 17: 0.0043, 18: 0.0026, 19: 0.0054, 20: 0.0009, 21: 0.0012, 22: 0.0003, 23: 0.001, 24: 0.0037, 25: 0.0029, 26: 0.0017, 29: 0.0019, 31: 0.0017, 33: 0.0003, 36: 0.0025, 38: 0.002, 39: 0.0018, 41: 0.0008, 42: 0.0003, 44: 0.0008, 45: 0.0019, 46: 0.0009, 47: 0.0007, 48: 0.0002, 49: 0.8813, 50: 0.0006, 51: 0.0001, 52: 0.0104, 53: 0.0116, 54: 0.0044, 55: 0.0001, 56: 0.0003, 57: 0.0131, 58: 0.0, 59: 0.0001, 60: 0.0001, 61: 0.0009, 62: 0.0006, 63: 0.001, 64: 0.0039})

In [0]:
output_col = vecAssembler.getOutputCol()
# Get the feature names and indices from the DataFrame's schema metadata
metadata = pipelineModel.transform(trainDF).select(output_col).schema[0].metadata["ml_attr"]["attrs"]
numeric_metadata = metadata["numeric"]
binary_metadata = metadata["binary"]

# Create a dictionary to map feature indices to names
index_to_name = {entry["idx"]: entry["name"] for entry in numeric_metadata + binary_metadata}

# Assuming gbt is your GBTRegressor model
feature_importances = pipelineModel.stages[-1].featureImportances

# Create a dictionary to map feature names to importances
feature_importance_dict = {index_to_name[i]: importance for i, importance in enumerate(feature_importances)}

# Order features by importance
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print sorted feature importances
for feature, importance in sorted_features:
    print(f"{feature}: {importance}")


AgeOHE_Ages 35-64 years: 0.8813034625032049
Year: 0.01311420854147841
RaceOHE_Hispanic: 0.011616048048561076
RaceOHE_Black (Non-Hispanic): 0.010426197009645298
StateOHE_MS: 0.005606385375256472
StateOHE_AL: 0.005443012167349257
StateOHE_TN: 0.004974871860181571
StateOHE_IA: 0.004576927350921709
RaceOHE_Asian/Pacific Islander: 0.00440382965497559
StateOHE_AR: 0.004284867810810718
StateOHE_MN: 0.004220794053921049
StateOHE_FL: 0.004075981014742899
StateOHE_KY: 0.0040266878462074775
average_embedding: 0.003931115576245422
StateOHE_CO: 0.0037138380664922433
StateOHE_SC: 0.0029307926938040687
StateOHE_NY: 0.0025923796346008767
StateOHE_NJ: 0.0025077853121173873
StateOHE_AZ: 0.0019601585621823155
StateOHE_CT: 0.0019131901597815777
StateOHE_NM: 0.001878031281562334
StateOHE_MA: 0.0017911818664598973
StateOHE_MT: 0.001724834394618227
StateOHE_NE: 0.001721885043751832
StateOHE_TX: 0.0017201886073924593
StateOHE_GA: 0.0015962860117874728
StateOHE_PA: 0.0012159227620360583
abs_compound: 0.0009876